<a href="https://colab.research.google.com/github/Raghav-hi/Google-colab/blob/main/bigDataAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing java and PySpark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packa

In [ ]:
#Verifying installation
import pyspark
print(pyspark.__version__)


3.5.5


In [ ]:
from google.colab import files
files.upload()  # Upload your kaggle.json file here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"raghavrajshrestha","key":"8e4b962448824480a3ff274d94153dd8"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [22]:
!kaggle datasets download -d prasad22/healthcare-dataset


Dataset URL: https://www.kaggle.com/datasets/prasad22/healthcare-dataset
License(s): CC0-1.0


In [23]:
!unzip healthcare-dataset.zip


Archive:  healthcare-dataset.zip
  inflating: healthcare_dataset.csv  


In [24]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("KaggleData").getOrCreate()

In [25]:
df = spark.read.csv("/content/healthcare_dataset.csv", header=True, inferSchema=True)
df.show(5)

+-------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|         Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|          Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|
+-------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|Bobby JacksOn| 30|  Male|        B-|           Cancer|       2024-01-31|   Matthew Smith|     Sons and Miller|        Blue Cross|18856.281305978155|        328|        Urgent|    2024-02-02|Paracetamol|      Normal|
| LesLie TErRy| 62|  Male|        A+|          Obesity|       2019-08-20| Samantha Davies|             Kim Inc|          Medicare|33

In [27]:
from pyspark.sql.functions import to_date, datediff, col

df = df.withColumn("Discharge Date", to_date(col("Discharge Date"), "MM/dd/yyyy")) \
       .withColumn("Date of Admission", to_date(col("Date of Admission"), "MM/dd/yyyy")) \
       .withColumn("Lenght_of_stay", datediff(col("Discharge Date"), col("Date of Admission")))

df.show(50)

+-------------------+---+------+----------+-----------------+-----------------+-----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|           Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|Lenght_of_stay|
+-------------------+---+------+----------+-----------------+-----------------+-----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------------+
|      Bobby JacksOn| 30|  Male|        B-|           Cancer|       2024-01-31|    Matthew Smith|     Sons and Miller|        Blue Cross|18856.281305978155|        328|        Urgent|    2024-02-02|Paracetamol|      Normal|             2|
|       LesLie TErRy| 62|  Male|        A+| 

In [29]:
#Data Cleaning
df = df.dropna()#Drop missing values:
from pyspark.sql.functions import col#Convert column types:
df = df.withColumn("Lenght_of_stay", col("Lenght_of_stay").cast("integer"))
#This checks if the column contains only numbers before converting.
#If the value is not numeric, it is replaced with NULL.

In [31]:
#Feature engineering
from pyspark.sql.functions import when, col

df = df.withColumn("Risk_Category",
                   when(col("Lenght_of_stay") > 3, "High")
                   .otherwise("Low"))#Adds Risk_Category feature
df.show(50)

+-------------------+---+------+----------+-----------------+-----------------+-----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------------+-------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|           Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|Lenght_of_stay|Risk_Category|
+-------------------+---+------+----------+-----------------+-----------------+-----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------------+-------------+
|      Bobby JacksOn| 30|  Male|        B-|           Cancer|       2024-01-31|    Matthew Smith|     Sons and Miller|        Blue Cross|18856.281305978155|        328|        Urgent|    2024-02-02|Paracetamol|      Normal|             2|   

In [32]:
rdd = df.rdd#RDD Approach
avg_length_of_stay = rdd.map(lambda row: row.Lenght_of_stay).mean()
print(f"Average Length of Stay (RDD): {avg_length_of_stay}")

Average Length of Stay (RDD): 15.509009009008967


In [37]:
df.agg({"Lenght_of_stay": "avg"}).show() # Changed "Length_of_stay" to "Lenght_of_stay"#DataFrame API

+-------------------+
|avg(Lenght_of_stay)|
+-------------------+
|  15.50900900900901|
+-------------------+



In [38]:
df.createOrReplaceTempView("patients")#SQL Approach
spark.sql("SELECT AVG(Lenght_of_stay) FROM patients").show()

+-------------------+
|avg(Lenght_of_stay)|
+-------------------+
|  15.50900900900901|
+-------------------+

